FACE RECOGNITION MODEL USING TENSORFLOW

# About the dataset #

Labeled Faces in the Wild (LFW) is a database of face photographs designed for studying the problem of unconstrained face recognition.
13,233 images of 5,749 people were detected and centered by the Viola Jones face detector and collected from the web.
1,680 of the people pictured have two or more distinct photos in the dataset.

Importing necessary dependencies

In [67]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [68]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [35]:
data_dir = 'archive/lfw-deepfunneled/lfw-deepfunneled'

Image preprocessing and data augmentation

In [38]:

datagen = ImageDataGenerator(
    rescale=1.0/255.0,  
    validation_split=0.2,  # 20% of data for validation
    horizontal_flip=True,  
    zoom_range=0.2, 
    shear_range=0.2, 
    rotation_range=20      
)

In [39]:
# Load data with data augmentation
train_data = datagen.flow_from_directory(
    directory=data_dir,
    target_size=(160, 160),  # Resize images to 160x160 pixels
    batch_size=32,  # Number of images per batch
    class_mode='categorical', 
    subset='training' 
)

Found 12161 images belonging to 5749 classes.


In [40]:
val_data = datagen.flow_from_directory(
    directory=data_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  
)

Found 1072 images belonging to 5749 classes.


In [ ]:
Using MobileNetV2 pre-trained model

In [42]:
base_model = MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')

In [43]:
# Freezing the base model 
base_model.trainable = False


In [44]:

x = base_model.output
x = GlobalAveragePooling2D()(x) 
x = Dense(1024, activation='relu')(x)  
predictions = Dense(train_data.num_classes, activation='softmax')(x)  

In [45]:
model = Model(inputs=base_model.input, outputs=predictions)

In [46]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 160, 160, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 80, 80, 32)        │             864 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 80, 80, 32)        │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 80, 80, 32)        │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 80, 80, 32)        │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 80, 80, 32)        │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 80, 80, 32)        │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 80, 80, 16)        │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 80, 80, 16)        │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 80, 80, 96)        │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 80, 80, 96)        │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 80, 80, 96)        │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 81, 81, 96)        │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 40, 40, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 9,462,453 (36.10 MB)

 Trainable params: 7,204,469 (27.48 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [49]:

history = model.fit(
    train_data,  # Training data
    epochs=10,  # Number of epochs to train for
    validation_data=val_data  # Validation data
)


Epoch 1/10


C:\DFR_project\frm\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


381/381 ━━━━━━━━━━━━━━━━━━━━ 363s 922ms/step - accuracy: 0.0335 - loss: 8.2345 - val_accuracy: 0.1250 - val_loss: 5.9655
Epoch 2/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 363s 944ms/step - accuracy: 0.0471 - loss: 7.3301 - val_accuracy: 0.1623 - val_loss: 5.3905
Epoch 3/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 355s 924ms/step - accuracy: 0.0614 - loss: 6.7924 - val_accuracy: 0.1810 - val_loss: 4.8613
Epoch 4/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 339s 884ms/step - accuracy: 0.0771 - loss: 6.2631 - val_accuracy: 0.2034 - val_loss: 4.6842
Epoch 5/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 344s 896ms/step - accuracy: 0.0976 - loss: 5.7474 - val_accuracy: 0.2192 - val_loss: 4.4919
Epoch 6/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 341s 887ms/step - accuracy: 0.1267 - loss: 5.1695 - val_accuracy: 0.2201 - val_loss: 4.5741
Epoch 7/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 337s 880ms/step - accuracy: 0.1704 - loss: 4.5208 - val_accuracy: 0.2211 - val_loss: 4.6197
Epoch 8/10
381/381 ━━━━━━━━━━━━━━━━━━━━ 329s 855ms/step - accuracy: 0.2254 - loss: 3.90

The training log shows that the accuracy and loss are improving with each epoch, but the improvement is slow.

Since the accuracy is improving, increasing the number of epochs might help the model converge better.

Validation Set Evaluation

In [51]:
val_loss, val_accuracy = model.evaluate(val_data)
print(f'Validation accuracy: {val_accuracy:.2f}')


34/34 ━━━━━━━━━━━━━━━━━━━━ 15s 440ms/step - accuracy: 0.1884 - loss: 5.1248
Validation accuracy: 0.19


In [73]:
def predict_identity(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))  
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0 
    img_array = np.expand_dims(img_array, axis=0)  
    prediction = model.predict(img_array)  
    predicted_class = np.argmax(prediction, axis=1)  
    
    return train_data.class_indices, predicted_class

class_indices, predicted_class = predict_identity('archive/lfw-deepfunneled/lfw-deepfunneled/Aaron_Eckhart/Aaron_Eckhart_0001.jpg')
predicted_person = list(class_indices.keys())[list(class_indices.values()).index(predicted_class)]
print(f'Predicted class: {predicted_person}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: Sheldon_Silver


We can enhance the data augmentation process to make the model more robust by using the albumentations library or we can
consider using a more complex model architecture or fine-tuning a pre-trained model like ResNet, or EfficientNet.